In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))
SOURCE_DIR
sys.path.insert(0, SOURCE_DIR)

In [3]:
%%time

import malaya
malaya

CPU times: user 3.08 s, sys: 3.17 s, total: 6.24 s
Wall time: 2.46 s


<module 'malaya' from '/home/husein/dev/malaya/malaya/__init__.py'>

In [4]:
normalizer = malaya.normalize.normalizer(date = False)

In [5]:
segmenter = malaya.segmentation.huggingface()
segmenter_func = lambda x: segmenter.generate([x], max_length = 256)[0]

In [6]:
%%time
segmenter_func('takkisah')

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


CPU times: user 677 ms, sys: 4.98 ms, total: 682 ms
Wall time: 64.2 ms


'tak kisah'

In [7]:
fasttext = malaya.language_detection.fasttext()
lang_word_model = malaya.language_detection.substring_rules(model = fasttext)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
from glob import glob

twitters = glob('/home/husein/malaya/2022-06-08/*.json')
twitters.extend(glob('/home/husein/malaya/compiled/*.json'))
twitters = sorted(twitters)

In [11]:
twitters

['/home/husein/malaya/2022-06-08/2022-06-08-bahasa-0.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-1.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-10.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-11.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-12.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-13.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-14.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-15.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-16.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-17.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-18.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-19.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-2.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-20.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-21.json',
 '/home/husein/malaya/2022-06-08/2022-06-08-bahasa-22.json',
 '/home/husein/malaya/2022-

In [12]:
rejected_words = ['sik', 'aktivitas', 'lanat', 'keren', 'loh', 'akh', 'pake', 'goblok', 'ivo',
                 'listrik', 'ludes', 'pemudik', 'stasiun', 'ketidaknyamananya', 'gawe', 'jempol',
                 'kesahatan', 'sivitas', 'ngasih', 'pasokan', 'esdm', 'miliar', 'domisili',
                 'nyengine', 'indonesia', 'ndak', 'mbak', 'gimana', 'verif']

In [13]:
import requests

def query(text, from_lang = 'en', to_lang = 'ms'):
    return requests.get(f'http://100.105.246.81:8999/api?from={from_lang}&to={to_lang}&text={text}').json()

In [14]:
text_scorer = lambda x: lm.score(x)

In [15]:
import re

def fix_spacing_punct(string):
    string = string.replace(' , ', ', ').replace(' !', '!').replace(' . ', '. ').replace('( ', '(') \
    .replace(' )', ')')
    return re.sub(r'[ ]+', ' ', string).strip()

In [18]:
from tqdm import tqdm
import traceback
import json

for f in twitters:
    filename = os.path.split(f)[1]
    output = f'{filename}.v2.normalized'
    output_requested = f"{filename.replace('.json', '')}.v2.normalized.jsonl.requested"
    print(filename, output, output_requested)
    
    if os.path.exists(output_requested):
        print(f'skip {filename}')
        continue
    
    with open(f) as fopen:
        data = json.load(fopen)

    results = []
    already_in = set()
    for row in tqdm(data):
        if row['retweet_text_full'] == 'NULL':
            t = row['data_text']
        else:
            t = row['retweet_text_full']

        if fasttext.predict([t])[0] not in ['malay', 'rojak']:
            continue

        if t.lower() in already_in:
            continue

        tokenized = normalizer._tokenizer(t.lower())
        if any([w in tokenized for w in rejected_words]):
            continue

        if not len(t):
            continue

        try:
            normalized = normalizer.normalize(
                t, normalize_hingga = False, normalize_cardinal = False,
                normalize_ordinal = False, normalize_pada_hari_bulan = False,
                normalize_fraction = False, normalize_money = False, normalize_date = False,
                normalize_time = False, normalize_ic = False,
                normalize_units = False,
                normalize_url = False, normalize_percent=False, normalize_telephone = False,
                not_a_word_threshold = 1e-9,
            )['normalize']
            
            normalized = fix_spacing_punct(normalized)
            
        except Exception as e:
            print(traceback.format_exc())
            print(e)
            normalized = t
        
        results.append({'original': t, 'normalized': normalized})
        already_in.add(t.lower())
    
    with open(output, 'w') as fopen:
        json.dump(results, fopen)

2022-06-08-bahasa-0.json 2022-06-08-bahasa-0.json.v2.normalized 2022-06-08-bahasa-0.v2.normalized.jsonl.requested


100%|███████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:38<00:00, 2625.30it/s]


2022-06-08-bahasa-1.json 2022-06-08-bahasa-1.json.v2.normalized 2022-06-08-bahasa-1.v2.normalized.jsonl.requested


100%|███████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:42<00:00, 2351.24it/s]


2022-06-08-bahasa-10.json 2022-06-08-bahasa-10.json.v2.normalized 2022-06-08-bahasa-10.v2.normalized.jsonl.requested


...

100%|███████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:34<00:00, 2861.06it/s]


2021-dump-bahasa-twitter-8.json 2021-dump-bahasa-twitter-8.json.v2.normalized 2021-dump-bahasa-twitter-8.v2.normalized.jsonl.requested


100%|███████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:38<00:00, 2572.11it/s]


2021-dump-bahasa-twitter-9.json 2021-dump-bahasa-twitter-9.json.v2.normalized 2021-dump-bahasa-twitter-9.v2.normalized.jsonl.requested


100%|███████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:37<00:00, 2671.38it/s]


In [ ]:
len(results)

In [ ]:
results

In [ ]:
fix_spacing_punct(results[5]['normalized'])

In [ ]:
normalizer._tokenizer('RM35.92 . Get')

In [ ]:
tokenizer = malaya.tokenizer.Tokenizer(parliament = True).tokenize

In [ ]:
tokenizer('RM35.92 . Get')

In [ ]:
tokenizer('RM35.92. Get')